# Mosaico 2021

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

In [102]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()
Map4 = geemap.Map()
Map5 = geemap.Map()

## Área de Estudio 

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

In [4]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [5]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [7]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [8]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [9]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [10]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [11]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10)) 

In [12]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2021

In [13]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [14]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [15]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [16]:
NDVI_2021 = coleccion2021.qualityMosaic('NDVI')

In [17]:
coleccion_sin_nubes = coleccion2021.map(Cloud20)
greenest_222 = coleccion_sin_nubes.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest_222, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [18]:
coleccion21 = coleccion2021.map(Cloud10)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [19]:
# coleccion21 = coleccion2021.map(Cloud10)
greenest21_v2 = coleccion2021.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [20]:
coleccion21_ = coleccion2021.map(Cloud30)
agua21 = coleccion21_.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [21]:
# coleccion21_ = coleccion2021.map(Cloud30)
agua21_v2 = coleccion2021.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [22]:
Map.addLayer(agua21, vizParams, 'agua 2021')
Map.addLayer(coleccion2021, vizParams, 'coleccion 2021')
Map.addLayer(agua21_v2, vizParams, 'agua v2')
Map.addLayer(greenest21, vizParams, 'greenest 2021')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2020

In [23]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [24]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [25]:
NDVI_2020 = coleccion2020.qualityMosaic('NDVI')

In [26]:
# coleccion20 = coleccion2020.map(Cloud10)
greenest20_v2 = coleccion2020.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20_v2, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [27]:
coleccion20 = coleccion2020.map(Cloud10)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [28]:
coleccion20_ = coleccion2020.map(Cloud30)
agua20 = coleccion20_.qualityMosaic('NDWI')
map_2020_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020_.add_ee_layer(agua20, vizParams, 'Mapa sin Nubes')
display(map_2020_)

### 2019

In [29]:
start_date = '2018-10-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [30]:
coleccion2019 = l8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [31]:
NDVI_2019 = coleccion2019.qualityMosaic('NDVI')

In [32]:
green19 = coleccion2019.map(Cloud30)
green19 = green19.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -70], zoom_start = 4)
map_sur.add_ee_layer(green19, vizParams, 'Mapa sin Nubes')
display(map_sur)

### 2018

In [73]:
start_date = '2017-10-01'
end_date = '2018-03-31'

l8_2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [74]:
coleccion2018 = l8_2018.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [75]:
NDVI_2018 = coleccion2018.qualityMosaic('NDVI')

In [76]:
green18 = coleccion2018.map(Cloud30)
green18 = green18.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -70], zoom_start = 4)
map_sur.add_ee_layer(green18, vizParams, 'Mapa sin Nubes')
display(map_sur)

### Año completo

In [33]:
start_date = '2018-10-01'
end_date = '2021-03-31'

l8_sur = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [34]:
coleccionsur = l8_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [35]:
greensur = coleccionsur.map(Cloud30)
greensur = greensur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -70], zoom_start = 4)
map_sur.add_ee_layer(greensur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [36]:
muestra_sur = greensur.sample(patagonia_grande, 10000)

In [37]:
options = {"title": 'Fechas Composición',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#1BD4C3']
          }

In [38]:
property = 'date'

In [39]:
chart.feature_histogram(muestra_sur, property, maxBuckets=30, **options)

## Análisis de agua

In [106]:
NDWI_2021 = agua21.select('NDWI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map5.addLayer(NDWI_2021, {'palette': palette}, 'nubes')
Map5

Map(bottom=22486.0, center=[-53.60554409923799, -70.6805867027383], controls=(WidgetControl(options=['position…

In [107]:
NDWI_img3 = img3.select('NDWI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map5.addLayer(NDWI_img3, {'palette': palette}, 'nubes')
Map5

Map(bottom=11355.0, center=[-53.10721669189339, -71.35707302761509], controls=(WidgetControl(options=['positio…

## Mosaico

In [40]:
nubes_agua = agua21_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map2.addLayer(nubes_agua, {'palette': palette}, 'nubes')

In [41]:
nubes21 = agua21.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map2.addLayer(nubes21, {'palette': palette}, 'nubes21')

In [42]:
Map2

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [43]:
nubes_greenest = greenest21.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest, {'palette': palette}, 'nubes1')

In [44]:
nubes_greenest21 = greenest21_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest21, {'palette': palette}, 'nubes2')

In [45]:
nubes_greenest20 = greenest20_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest20, {'palette': palette}, 'nubes3')

In [46]:
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [47]:
def llenado(image):
    return image.where(nubes_greenest21.gt(30),greenest20)

In [48]:
img1 = llenado(greenest21_v2)

In [49]:
map_img1 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img1.add_ee_layer(img1, vizParams, 'Mapa sin Nubes')
display(map_img1)

In [50]:
# mosaico1 =  img1.set('SENSOR_ID', 'OLI_TIRS')


In [51]:
type(coleccion2021)

ee.imagecollection.ImageCollection

In [52]:
type(greenest21)

ee.image.Image

In [53]:
type(img1)

ee.image.Image

In [54]:
corregido = img1.set('SENSOR_ID', 'OLI_TIRS')

In [55]:
scored_mosaico= ee.Algorithms.Landsat.simpleCloudScore(corregido)
# img_nubes_mosaico = ee.Image(scored_mosaico)

In [56]:
type(scored_mosaico)

ee.image.Image

In [57]:
map_nube = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube.add_ee_layer(scored_mosaico, vizParams , 'Mapa sin Nubes')
display(map_nube)

In [58]:
nubes_mosaico= scored_mosaico.select('cloud')
map_nube = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube.add_ee_layer(nubes_mosaico, {'palette': palette} , 'Mapa sin Nubes')
display(map_nube)

In [59]:
# def llenado2(image):
#     return image.where(nubes_greenest21.eq(100),agua21)

In [60]:
# img2 = llenado2(img1) 

In [61]:
# map_img2 = folium.Map(location = [-45, -70], zoom_start = 4)
# map_img2.add_ee_layer(img2, vizParams, 'Mapa sin Nubes')
# display(map_img2)

In [62]:
def llenado3(image):
    return image.where(nubes_mosaico.gt(30),green19)

In [63]:
img2 = llenado3(img1) 

In [64]:
map_img2 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img2.add_ee_layer(img2, vizParams, 'Mapa sin Nubes')
display(map_img2)

In [65]:
Map4.addLayer(greenest21_v2, vizParams, '0')
Map4.addLayer(img1, vizParams, '1')
Map4.addLayer(img2, vizParams, '2')
Map4

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [69]:
corregido2 = img2.set('SENSOR_ID', 'OLI_TIRS')

In [70]:
scored_mosaico2 = ee.Algorithms.Landsat.simpleCloudScore(corregido2)

In [71]:
map_nube2 = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube2.add_ee_layer(scored_mosaico2, vizParams , 'Mapa sin Nubes')
display(map_nube2)

In [72]:
nubes_mosaico2 = scored_mosaico2 .select('cloud')
map_nube2 = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube2.add_ee_layer(nubes_mosaico2, {'palette': palette} , 'Mapa sin Nubes')
display(map_nube2)

In [78]:
def llenado4(image):
    return image.where(nubes_mosaico2.gt(30),green18)

In [79]:
img3 = llenado4(img2) 

In [80]:
map_img3 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img3.add_ee_layer(img3, vizParams, 'Mapa sin Nubes')
display(map_img3)

In [129]:
def llenado5(image):
    return image.where(NDWI_img3.gt(-0.1),agua21)

In [130]:
img4 = llenado5(img3)

In [131]:
map_img4 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img4.add_ee_layer(img4, vizParams, 'Mapa sin Nubes')
display(map_img4)

In [81]:
options = {"title": 'Fechas Composición',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#14A5BB']
          }

In [82]:
muestra_mosaico = img3.sample(patagonia_grande, 10000)

In [83]:
chart.feature_histogram(muestra_mosaico, property, maxBuckets=30, **options)

In [100]:
Map4.addLayer(greenest21_v2, vizParams, '0')
Map4.addLayer(img1, vizParams, '1')
Map4.addLayer(img2, vizParams, '2')
Map4.addLayer(img2, vizParams, '3')
Map4

Map(bottom=5882.0, center=[-54.52108149544361, -71.08154296875001], controls=(WidgetControl(options=['position…

### Visualización

In [85]:
resultados = [greenest21_v2, img1, img2, img3]

In [86]:
labels = [
    'Greenest 2021',
    'Mosaico Veranos 2020-2021',
    'Moasico Veranos 2019-2021',
    'Mosaico Veranos 2018-2021', 
]

In [98]:
vis_params2 = [
    vizParams, 
    vizParams,
    vizParams,
    vizParams,
]

In [95]:
vis_nieve = {
  'bands': ['B6', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [97]:
vis_params3 = [
    vis_nieve, 
    vis_nieve,
    vis_nieve,
    vis_nieve,
]

#### Visualización 1

In [92]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

#### Visualización 2

In [93]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

#### Visualización 3

In [94]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [99]:
geemap.linked_maps(rows=2, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…